In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('max_column', 100)

In [2]:
raw_path = '../../dataset/raw/'
mst_path = '../../dataset/mst/'

In [3]:
os.listdir(raw_path)

['Participant_Survey.xlsx']

In [5]:
df = pd.read_excel(raw_path + "Participant_Survey.xlsx")

In [6]:
df

,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,First Name,Last Name,Custom Data 1,"What brand/type of E-bike did you buy (e.g., Sonders X)?",How much did your E-bike cost (before the rebate)?,"Overall, how satisfied are you with your E-bike?","Why do you like or dislike your E-bike? What, if any, other thoughts would you like to share about your E-bike?",How much did the $700 rebate influence your decision to purchase an E-bike?,How far is your average trip by E-bike?,For what purpose are MOST of your E-bike rides?,Why did you purchase an E-bike? Select all that apply.,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,How did you hear about the E-bike rebate program? Select all that apply.,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,How familiar were you with E-bikes before hearing about this rebate program?,Which of the following do you have at home? (Select all that apply),Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Select your top 3 most-used modes of transportation BEFORE purchasing an E-Bike.,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Select your top 3 most-used modes of transportation AFTER purchasing an E-Bike.,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,How often did you bicycle for any purpose BEFORE purchasing an E-bike?,How often do you bicycle for any purpose SINCE purchasing an E-bike?,Do you use your E-bike for some trips that you used to take by car?,"Have you used your E-bike on a river path or ‘greenway’ in the past 12 months? (e.g., San Gabriel River or Rio Hondo River trails)",How easy or difficult is it to bicycle in your community?,Did your level of comfort riding a bicycle in traffic increase after taking the City Cycling Class?,How safe do you feel while using the following types of bike facilities? (See images below for reference),Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,How safe do you feel riding a bicycle in traffic?,"What barriers to using your E-bike have you encountered thus far, if any?",How many members of your household use your E-bike at least once a month?,"Have you allowed any friends, family, colleagues, neighbors, or others to try using your E-bike? If so, how many individuals other than you?",Unnamed: 69,Would you like to see future E-bike rebate programs?,Should future programs expand the list of eligible light electric vehicles to include electric-scooters and similar micro-mobility devices?,"Other suggestions, feedback, or comments for project organizers.",Community of Residence,Zip Code,Number of persons in household,Household income,What is your age group?,"Do you own any other Electric Vehicles (EVs)? If so, please select all that apply.",Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Ethnicity/Race (select all that apply),Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,What gender do you identify with?,Unnamed: 94,Unnamed: 95,Unnamed: 96
0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,Open-Ended Response,Response,Response,Open-Ended Response,Response,Response,Response,Business/Work Use,Injury or Disability,Sustainability / Environmental Benefits,Cargo Hauling,Recreation or Exercise,Transport children,Save money / reduce transportation costs,Hills,Other (please specify),"Social Media (e.g., Facebook, Instagram, Twitter)",News article,BikeSGV listserve/email,Friend/family/colleague,Flyer,Poster,Other (please specify),Response,Bicycle (regular),Electric Bicycle (E-bike),Working motor vehicle,Scooter,Motorcycle,None of the above,Walked,Biked,Skated / Scooted,Used public transit,Carpooled / rode as a passenger,Used ride share service (Lyft/Uber),Drove car / motorcycle,Walked,Biked,Skated / Scooted,Used public transit,Carpooled / rode as a passenger,Used ride share service (Lyft/Uber),Drove car / motorcycle,Response,Response,Response,Response,Respon